In [49]:
import pandas as pd
import numpy as np

import geopandas as gpd
import geopandas.tools as gdp_tools
from shapely.geometry import shape, Point, Polygon
import fiona

import json

In [50]:
DATA_LOC ='./data/'
DATA_BASE ='./data/'

In [51]:
file = DATA_LOC + 'Basisdata_0000_Norge_25833_MatrikkelenAdresse_CSV' + '/' + 'matrikkelenAdresse.csv'

In [55]:
df_csv = pd.read_csv(file, encoding='utf-8', sep=';')

/Users/paulbencze/Projects/maps/.venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (4,5,7,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [56]:
df_csv.head()

,lokalid,kommunenummer,kommunenavn,adressetype,adressetilleggsnavn,adressetilleggsnavnKilde,adressekode,adressenavn,nummer,bokstav,...,atkomstNord,atkomstØst,sommeratkomstId,uuidSommeratkomst,sommeratkomstNord,sommeratkomstØst,vinteratkomstId,uuidVinteratkomst,vinteratkomstNord,vinteratkomstØst
0,216281051,3050,FLESBERG,matrikkeladresse,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,216281014,3050,FLESBERG,matrikkeladresse,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,216281015,3050,FLESBERG,matrikkeladresse,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,216281021,3050,FLESBERG,matrikkeladresse,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,216279909,3050,FLESBERG,matrikkeladresse,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
df_csv.dtypes

lokalid                                  int64
kommunenummer                            int64
kommunenavn                             object
adressetype                             object
adressetilleggsnavn                     object
adressetilleggsnavnKilde                object
adressekode                            float64
adressenavn                             object
nummer                                 float64
bokstav                                 object
gardsnummer                              int64
bruksnummer                              int64
festenummer                            float64
undernummer                            float64
adresseTekst                            object
adresseTekstUtenAdressetilleggsnavn     object
EPSG-kode                                int64
Nord                                   float64
Øst                                    float64
postnummer                               int64
poststed                                object
grunnkretsnum

In [59]:
df_csv[['kommunenummer',
        'kommunenavn',
        'adressetype', 
        'gardsnummer',
        'bruksnummer',
        'festenummer',
        'undernummer',
        'Nord',
        'Øst'
       ]]

,kommunenummer,kommunenavn,adressetype,gardsnummer,bruksnummer,festenummer,undernummer,Nord,Øst
0,3050,FLESBERG,matrikkeladresse,12,88,NaN,NaN,6643517.98,185665.36
1,3050,FLESBERG,matrikkeladresse,12,85,NaN,NaN,6643742.24,185686.73
2,3050,FLESBERG,matrikkeladresse,12,86,NaN,NaN,6643555.24,185664.72
3,3050,FLESBERG,matrikkeladresse,95,1,63.0,NaN,6652530.09,193745.56
4,3050,FLESBERG,matrikkeladresse,131,9,NaN,NaN,6638120.49,199649.05
...,...,...,...,...,...,...,...,...,...
2541836,5426,KÅFJORD,vegadresse,32,2,1.0,NaN,7707520.00,718542.00
2541837,3453,ØYSTRE SLIDRE,vegadresse,30,12,5.0,NaN,6825823.98,172494.53
2541838,5035,STJØRDAL,vegadresse,180,5,NaN,NaN,7037691.89,295156.48
2541839,5035,STJØRDAL,vegadresse,220,13,NaN,NaN,7038413.24,311747.42


In [62]:
df_csv['Coordinates'] = list(zip(df_csv.Nord, df_csv.Øst))

In [64]:
df_csv['Coordinates'] = df_csv['Coordinates'].apply(Point)

In [66]:
gdf = gpd.GeoDataFrame(df_csv, geometry='Coordinates')

In [67]:
gdf.crs = dict(ellps='WGS84', datum='WGS84', proj='utm', zone=33, units='m', no_defs=True)
gdf = gdf.to_crs(dict(ellps='WGS84', datum='WGS84', proj='lonlat', units='km', no_defs=True))

In [68]:
gdf.head()

,lokalid,kommunenummer,kommunenavn,adressetype,adressetilleggsnavn,adressetilleggsnavnKilde,adressekode,adressenavn,nummer,bokstav,...,atkomstØst,sommeratkomstId,uuidSommeratkomst,sommeratkomstNord,sommeratkomstØst,vinteratkomstId,uuidVinteratkomst,vinteratkomstNord,vinteratkomstØst,Coordinates
0,216281051,3050,FLESBERG,matrikkeladresse,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (63.20057 1.11505)
1,216281014,3050,FLESBERG,matrikkeladresse,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (63.20191 1.11515)
2,216281015,3050,FLESBERG,matrikkeladresse,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (63.20079 1.11504)
3,216281021,3050,FLESBERG,matrikkeladresse,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (63.25538 1.16233)
4,216279909,3050,FLESBERG,matrikkeladresse,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (63.17026 1.19978)


In [70]:
gdf.dtypes

lokalid                                   int64
kommunenummer                             int64
kommunenavn                              object
adressetype                              object
adressetilleggsnavn                      object
adressetilleggsnavnKilde                 object
adressekode                             float64
adressenavn                              object
nummer                                  float64
bokstav                                  object
gardsnummer                               int64
bruksnummer                               int64
festenummer                             float64
undernummer                             float64
adresseTekst                             object
adresseTekstUtenAdressetilleggsnavn      object
EPSG-kode                                 int64
Nord                                    float64
Øst                                     float64
postnummer                                int64
poststed                                

In [46]:
file = DATA_LOC + 'Basisdata_0000_Norge_25833_MatrikkelenBygning_FGDB.gdb'

bygninger = []
for i, name in enumerate(fiona.listlayers(file)):
    print(name)
    if name == 'matrikkelenbygning_bygning':
        with fiona.open(file, layer=i) as layer:
            index = 0
            for feature in layer:
                index = index + 1
                objtype = (feature['properties'])
                #if objtype != 'Bygning':
                bygninger.append(feature['properties'])
                if index > 0: 
                    break

matrikkelenbygning_bygning
matrikkelenbygning_sefrakident


In [47]:
bygninger

[OrderedDict([('objtype', 'Bygning'),
              ('bygningstype', 161),
              ('bygningsnummer', 174256527),
              ('bygningsstatus', 'TB'),
              ('harkulturminne', 0),
              ('harsefrakminne', 0),
              ('opprinnelse', 'Massivregistreringer'),
              ('kommune', '4616'),
              ('kommunenavn', 'TYSNES'),
              ('lokalid', '192640544'),
              ('navnerom',
               'http://skjema.geonorge.no/SOSI/produktspesifikasjon/Matrikkelen-Bygningspunkt/20180501'),
              ('versjonid', '3'),
              ('datauttaksdato', '2022-02-26T09:34:18+00:00'),
              ('oppdateringsdato', '2020-06-15T19:15:35+00:00'),
              ('stedfestingverifisert', 0)])]

In [60]:
file = DATA_LOC + 'Basisdata_0000_Norge_25833_MatrikkelenAdresse_FGDB.gdb'

bygninger = []
for i, name in enumerate(fiona.listlayers(file)):
    print(name)
    if name == 'vegadresse':
        with fiona.open(file, layer=i) as layer:
            index = 0
            for feature in layer:
                index = index + 1
                objtype = (feature['properties'])
                #if objtype != 'Bygning':
                bygninger.append(feature['properties'])
                if index > 0: 
                    break

matrikkeladresse
vegadresse
matrikkeladresse_atkomstid
atkomst
vegadresse_atkomstid


In [61]:
bygninger

[OrderedDict([('objtype', 'VegadresseBruksenhet'),
              ('adressenavn', 'Gampeflåvegen'),
              ('bokstav', None),
              ('nummer', 40),
              ('adresseid', '216281038'),
              ('navnerom', 'https://data.geonorge.no/sosi/matrikkel/adresse'),
              ('versjonid', '4'),
              ('oppdateringsdato', '2020-06-15T17:32:59+00:00'),
              ('datauttaksdato', '2022-02-28T01:13:39+00:00'),
              ('stedfestingverifisert', 0),
              ('adressetekst', 'Gampeflåvegen 40'),
              ('adressetekstutenadressetilleggsnavn', 'Gampeflåvegen 40'),
              ('adressetilleggsnavn', None),
              ('adressetilleggsnavnkilde', None),
              ('grunnkretsnummer', '30500106'),
              ('grunnkretsnavn', 'GÅSUM'),
              ('kommunenummer', '3050'),
              ('kommunenavn', 'FLESBERG'),
              ('matrikkelnummerbruksenhet_kommunenummer', '3050'),
              ('matrikkelnummerbruksenhet_gards

In [ ]:
class jsonEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Point):
            obj = [obj.coords[0][0], obj.coords[0][1]]
        else:
            obj = super(jsonEncoder, self).default(obj)
        return obj

In [ ]:
tg = json.loads(bg_combgm.to_json(cls=jsonEncoder))
path = DATA_LOC + 'counties_2020_geojson.json'
with open(path, 'w') as fp:
    json.dump(tg, fp)

In [ ]:
simplify('counties_2020')

In [ ]:
%matplotlib inline

import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (16,9)
import matplotlib.pyplot as plt